In [38]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [39]:
from btc_model.core.util.crypto_util import CryptoUtil

In [2]:
import ccxt
from btc_model.setting.setting import get_settings

# 获取设置
setting = get_settings('cex.okx')

apikey = setting['apikey']
secretkey = setting['secretkey']
passphrase = setting['passphrase']
proxy = setting['proxy']

# 初始化币安交易所
params = {
    'enableRateLimit': True,
    'proxies': {
        'http': get_settings('common')['proxies']['http'] if proxy else None,                  
        'https': get_settings('common')['proxies']['https'] if proxy else None,
    },
    'apiKey': apikey,          
    'secret': secretkey,  
    'password': passphrase,     
    'options': {
        'defaultType': 'spot',  # 可选：'spot', 'margin', 'future'
    },
    'headers': {
        'x-simulated-trading': '1'
    }
}

exchange = ccxt.okx(params)

exchange.set_sandbox_mode(True)


In [14]:
try:
    # 获取账户余额
    balance = exchange.fetch_balance({
        'type': 'trading'
    })
    
    print("=== 模拟账户余额 ===")
    for currency in balance['total']:
        if balance['total'][currency] > 0:
            print(f"{currency}:")
            print(f"  可用: {balance['free'][currency]}")
            print(f"  冻结: {balance['used'][currency]}")
            print(f"  总额: {balance['total'][currency]}")

except Exception as e:
    print(f"错误: {str(e)}")


=== 模拟账户余额 ===
BTC:
  可用: 1.0
  冻结: 0.0
  总额: 1.0
USDT:
  可用: 5000.0
  冻结: 0.0
  总额: 5000.0
OKB:
  可用: 100.0
  冻结: 0.0
  总额: 100.0
ETH:
  可用: 1.0
  冻结: 0.0
  总额: 1.0


In [13]:

try:
    # 现货限价单
    def place_spot_limit_order(symbol, side, amount, price):
        """
        symbol: 交易对，如 'BTC/USDT'
        side: 'buy' 或 'sell'
        amount: 数量
        price: 价格
        """
        order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side=side,
            amount=amount,
            price=price
        )
        return order

    # 现货市价单
    def place_spot_market_order(symbol, side, amount):
        order = exchange.create_order(
            symbol=symbol,
            type='market',
            side=side,
            amount=amount
        )
        return order

    # 永续合约限价单
    def place_swap_limit_order(symbol, side, amount, price, leverage=1):
        """
        symbol: 交易对，如 'BTC/USDT:USDT'
        side: 'buy' 或 'sell'
        amount: 合约数量
        price: 价格
        leverage: 杠杆倍数
        """
        # 设置杠杆
        exchange.set_leverage(leverage, symbol)
        
        order = exchange.create_order(
            symbol=symbol,
            type='limit',
            side=side,
            amount=amount,
            price=price,
            params={
                'tdMode': 'cross',  # 全仓模式，'isolated' 为逐仓模式
                'posSide': 'long'   # 持仓方向，'long' 或 'short'
            }
        )
        return order


    # 示例使用
    # 现货限价买入
    min_amount = 0.01  # 假设最小交易量为 0.01
    spot_order = place_spot_limit_order(
        symbol='BTC/USDT',
        side='buy',
        amount=min_amount,  # 使用最小交易量
        price=40000         # 价格 40000 USDT
    )
    print("现货限价单:", spot_order)

    # 永续合约开多
    swap_order = place_swap_limit_order(
        symbol='BTC/USDT:USDT',
        side='buy',
        amount=min_amount,  # 使用最小交易量
        price=40000,
        leverage=2          # 2倍杠杆
    )
    print("永续合约限价单:", swap_order)

except Exception as e:
    print(f"下单错误: {str(e)}")

# 查询订单状态
def check_order_status(order_id, symbol):
    try:
        order = exchange.fetch_order(order_id, symbol)
        print(f"订单状态: {order['status']}")
        return order
    except Exception as e:
        print(f"查询订单错误: {str(e)}")
        return None

# 取消订单
def cancel_order(order_id, symbol):
    try:
        result = exchange.cancel_order(order_id, symbol)
        print("订单已取消")
        return result
    except Exception as e:
        print(f"取消订单错误: {str(e)}")
        return None

现货限价单: {'info': {'clOrdId': 'e847386590ce4dBC141ae60e0dd45f79', 'ordId': '2291432507889983488', 'sCode': '0', 'sMsg': 'Order placed', 'tag': 'e847386590ce4dBC', 'ts': '1740792410329'}, 'id': '2291432507889983488', 'clientOrderId': 'e847386590ce4dBC141ae60e0dd45f79', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'limit', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'triggerPrice': None, 'average': None, 'cost': None, 'amount': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'reduceOnly': False, 'fees': [], 'stopPrice': None}
下单错误: okx {"code":"1","data":[{"clOrdId":"e847386590ce4dBCa9ceb4399b2cbf5a","ordId":"","sCode":"51010","sMsg":"You can't complete this request under your current account mode. ","tag":"e847386590ce4dBC","ts":"1740792410713"}],"inTime":"1740792410713393","msg":"All operations failed",

In [14]:
# 现货市价单
def place_spot_market_order(symbol, side, amount):
    order = exchange.create_order(
        symbol=symbol,
        type='market',
        side=side,
        amount=amount
    )
    return order

# 示例使用
try:
    # 使用市价单买入
    market_order = place_spot_market_order(
        symbol='BTC/USDT',
        side='buy',
        amount=0.01  # 确保数量符合最小交易量
    )
    print("现货市价单:", market_order)

except Exception as e:
    print(f"下单错误: {str(e)}")

现货市价单: {'info': {'clOrdId': 'e847386590ce4dBCbeca8fa4f69eac40', 'ordId': '2291433857851891712', 'sCode': '0', 'sMsg': 'Order placed', 'tag': 'e847386590ce4dBC', 'ts': '1740792450561'}, 'id': '2291433857851891712', 'clientOrderId': 'e847386590ce4dBCbeca8fa4f69eac40', 'timestamp': None, 'datetime': None, 'lastTradeTimestamp': None, 'lastUpdateTimestamp': None, 'symbol': 'BTC/USDT', 'type': 'market', 'timeInForce': None, 'postOnly': None, 'side': 'buy', 'price': None, 'stopLossPrice': None, 'takeProfitPrice': None, 'triggerPrice': None, 'average': None, 'cost': None, 'amount': None, 'filled': None, 'remaining': None, 'status': None, 'fee': None, 'trades': [], 'reduceOnly': False, 'fees': [], 'stopPrice': None}


In [23]:
markets = exchange.load_markets()

In [26]:
markets_b = {
            (m['base'], m['quote']): m['symbol']
            for m in exchange.markets.values()
            if m['type'] == 'spot' 
        }

In [27]:
markets_b

{('SOL', 'BTC'): 'SOL/BTC',
 ('MKR', 'BTC'): 'MKR/BTC',
 ('NC', 'BTC'): 'NC/BTC',
 ('QTUM', 'BTC'): 'QTUM/BTC',
 ('SKL', 'BTC'): 'SKL/BTC',
 ('CRO', 'OKB'): 'CRO/OKB',
 ('KLAY', 'OKB'): 'KLAY/OKB',
 ('BTC', 'SGD'): 'BTC/SGD',
 ('USDT', 'SGD'): 'USDT/SGD',
 ('USDC', 'SGD'): 'USDC/SGD',
 ('BTC', 'USD'): 'BTC/USD',
 ('DOGE', 'USD'): 'DOGE/USD',
 ('BIO', 'USD'): 'BIO/USD',
 ('OP', 'USD'): 'OP/USD',
 ('USDT', 'USD'): 'USDT/USD',
 ('USDT', 'AUD'): 'USDT/AUD',
 ('BTC', 'AED'): 'BTC/AED',
 ('ETH', 'AED'): 'ETH/AED',
 ('USDT', 'AED'): 'USDT/AED',
 ('BTC', 'HKD'): 'BTC/HKD',
 ('ETH', 'HKD'): 'ETH/HKD',
 ('AVAX', 'HKD'): 'AVAX/HKD',
 ('AXS', 'HKD'): 'AXS/HKD',
 ('LINK', 'HKD'): 'LINK/HKD',
 ('USDT', 'HKD'): 'USDT/HKD',
 ('BTC', 'BRL'): 'BTC/BRL',
 ('ETH', 'BRL'): 'ETH/BRL',
 ('SOL', 'BRL'): 'SOL/BRL',
 ('USDT', 'BRL'): 'USDT/BRL',
 ('BTC', 'EUR'): 'BTC/EUR',
 ('ETH', 'EUR'): 'ETH/EUR',
 ('XRP', 'EUR'): 'XRP/EUR',
 ('SOL', 'EUR'): 'SOL/EUR',
 ('AGIX', 'EUR'): 'AGIX/EUR',
 ('APT', 'EUR'): 'APT/EUR'

In [15]:
import ccxt


# 获取OKX的永续合约交易对
perpetual_pairs = [market['symbol'] for market in exchange.markets.values() if market['type'] == 'swap' and market['linear']]

print(perpetual_pairs)


['BTC/USDT:USDT', 'ETH/USDT:USDT', 'LTC/USDT:USDT', 'XRP/USDT:USDT', 'SOL/USDT:USDT', 'DOGE/USDT:USDT', 'ADA/USDT:USDT', '1INCH/USDT:USDT', 'AAVE/USDT:USDT', 'ACT/USDT:USDT', 'AI16Z/USDT:USDT', 'AIDOGE/USDT:USDT', 'ALGO/USDT:USDT', 'ARB/USDT:USDT', 'ATOM/USDT:USDT', 'AUCTION/USDT:USDT', 'AVAX/USDT:USDT', 'AXS/USDT:USDT', 'BAL/USDT:USDT', 'BAND/USDT:USDT', 'BCH/USDT:USDT', 'BNB/USDT:USDT', 'BNT/USDT:USDT', 'BOME/USDT:USDT', 'BRETT/USDT:USDT', 'CATI/USDT:USDT', 'CAT/USDT:USDT', 'CELO/USDT:USDT', 'CETUS/USDT:USDT', 'CFX/USDT:USDT', 'COMP/USDT:USDT', 'CRO/USDT:USDT', 'CRV/USDT:USDT', 'DOGS/USDT:USDT', 'DOT/USDT:USDT', 'EOS/USDT:USDT', 'ETC/USDT:USDT', 'ETHW/USDT:USDT', 'ETHFI/USDT:USDT', 'FIL/USDT:USDT', 'FLOKI/USDT:USDT', 'FLOW/USDT:USDT', 'GMT/USDT:USDT', 'HBAR/USDT:USDT', 'HMSTR/USDT:USDT', 'ICP/USDT:USDT', 'ID/USDT:USDT', 'IMX/USDT:USDT', 'INJ/USDT:USDT', 'IOTA/USDT:USDT', 'KISHU/USDT:USDT', 'KNC/USDT:USDT', 'KSM/USDT:USDT', 'LDO/USDT:USDT', 'LINK/USDT:USDT', 'LPT/USDT:USDT', 'LQTY/USD

In [20]:
symbol = 'LSK/USDT:USDT'
limit = 30
history = exchange.fetch_funding_rate_history(symbol, limit=limit)

BadSymbol: okx does not have market symbol LSK/USDT:USDT

In [ ]:
from btc_model.core.util.crypto_util import CryptoUtil
crypto_util = CryptoUtil()
symbol = 'BTC/USDT:USDT'
crypto_util.get_ohlcv(exchange, symbol)

In [37]:
crypto_util.get_withdrawal_fees(exchange, ['BTC'])

Exception: 获取提现费率失败: okx {"code":"50038","data":[],"msg":"This feature is unavailable in demo trading"}